# Ejercicio: web scraping con Python

En este ejercicio vas a hacer [scraping](https://es.wikipedia.org/wiki/Web_scraping) de unas cuantas páginas de productos de [Amazon España](https://www.amazon.es/).

El scraping consiste en utilizar una herramienta informática (como puede ser un lenguaje de programación) para extraer datos de una página web de forma automática. Básicamente utiliza [peticiones HTTP](https://es.wikipedia.org/wiki/Hypertext_Transfer_Protocol) para "pedir" una página web de forma similar a como haríamos con un navegador web (como Firefox, Chrome o Internet Explorer). Una vez hecha dicha petición, extrae la información que nos interesa y la guarda (en un archivo o en una base de datos).

Lo primero que necesitamos es una biblioteca capaz de hacer estas peticiones HTTP, es decir: que se conecte a la página web que queremos, y nos "traiga" a Python el contenido de dicha web.

Para este ejercicio vamos a utilizar la página web de Amazon de esta bicicleta de montaña: https://www.amazon.es/Moma-Bicicleta-Mountainbike-aluminio-suspensi%C3%B3n/dp/B00VXE2JCY/ref=lp_2929469031_1_1?s=sports&ie=UTF8&qid=1465237289&sr=1-1&th=1

## Pide HTTP con `Requests`

Python incluye en la biblioteca estándar algunas utilidades para hacer peticiones HTTP. Un ejemplo es [`urllib`](https://docs.python.org/3/library/urllib.html). No obstante, se trata de una biblioteca bastante complicada para algo tan sencillo como pedir páginas web. 

Por eso mismo vamos a utilizar una biblioteca mucho más sencilla y potente: [`requests`](http://docs.python-requests.org/en/master/). Requests no es parte de la bilbioteca estándar, sino que debe ser instalada. No obstante, la distribución de Python Anaconda (la que utilizamos para este curso) la trae preinstalada; así que no tenemos que hacerlo.

De todos modos es muy fácil de instalar. No tendríamos más que hacer:

```sh
$ pip install requests
```

La [documentación de Requests](http://docs.python-requests.org/en/master/#the-user-guide) es explícita, liviana y entretenida de leer. Sin más dilación, importemos la biblioteca:

In [1]:
# Importa aquí la bilbioteca requests:
import requests

Una vez importada, Requests debe "pedir" una página web (en este caso https://www.amazon.es/Moma-Bicicleta-Mountainbike-aluminio-suspensi%C3%B3n/dp/B00VXE2JCY/ref=lp_2929469031_1_1?s=sports&ie=UTF8&qid=1465237289&sr=1-1&th=1). Solo tenemos que decirle a Requests que haga lo que se llama *una petición HTTP GET*. Los dos tipos básicos de peticiones en web son las [HTTP GET y las HTTP POST](http://www.w3schools.com/tags/ref_httpmethods.asp). En este caso, puesto que no queremos "subir" nada, sino recibir, haremos una HTTP GET con Requests.

Además, ciertas peticiones deben llevar algo de información adicional, lo que se conoce como [cabeceras](https://es.wikipedia.org/wiki/Cabeceras_HTTP) o *headers*, que son pares clave-valor. Para el caso de Amazon España, necesitamos decirle a sus servidores web que *somos un navegador normal y corriente, y no la biblioteca `requests` de Python*. Si no lo hacemos, Amazon nos bloqueará el contenido :(

Para ello, necesitamos añadir una cabecera que sea de la siguiente forma: como clave `"User-Agent"`, y como valor `"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:56.0) Gecko/20100101 Firefox/56.0"` (para *hacernos pasar* por el Firefox de un Mac).

*Importante:* A fecha del 9 de septiembre de 2017, Amazon ha estado trabajando duro durante el verano y ha mejorado bastante su modelo de machine learning de identificación de bots y scrappers (antes era una patata y muy fácil de engañar). Si crees que estás haciendo bien el ejercicio, pero que no consigues extraer el texto (y que en los datos extraídos salen cosas raras como *Teclea los caracteres que aparecen en la imagen*), es muy posible que Amazon crea (correctamente) que le estamos intentando scrapear. Hay varias técnicas para intentar engañarle: apagar y encender el wifi en el portátil (para re-conectarnos a Internet), ir cambiando el *User Agent* que utilizamos (en vez del `"Firefox/56.0"` podemos probar `"Firefox/55.0"`, `"Chrome/51.0.2704.103"`, `"PepitoDeLosPalotes"`... Cualquier cosa vale), copiarte el Notebook y ejecutarlo desde otro ordenador o red... Si ves que te da muchísimos problemas y que no consigues que Amazon te bendiga con su visto bueno, no dudes en escribirme.

En la [quickstart](http://docs.python-requests.org/en/master/user/quickstart/#make-a-request) puedes informarte sobre lo sencillo que es hacer una petición GET con Requests. También tendrás que mirar cómo añadir tu *header* a la petición. Al lío:

In [ ]:
# Haz una petición GET de la url de la bicicleta, con el 
# header especificado, y asigna el resultado en una variable 
# llamada peticion_bicicleta:


Vamos a ver el tipo de nuestra variable `peticion_bicicleta`:

In [ ]:
# Imprime con un print el tipo de dato de
# la variable peticion_bicicleta:


Es un tipo de dato (una instancia) de la clase [`Response`](http://docs.python-requests.org/en/master/api/#requests.Response). 

Cuando intentamos ver una página web (o dicho de otro modo: hacemos una petición HTTP a dicha página), el servidor de dicha web nos devuelve un [código HTTP de respuesta](https://es.wikipedia.org/wiki/Anexo:C%C3%B3digos_de_estado_HTTP). Toca buscar en la documentación si las instancias de la clase `Response` tienen algún atributo (*class*/*instance variable*) o método para saber si la petición era adecuada, y si el contenido nos ha sido devuelto correctamente...

In [ ]:
# Imprime con un print el código HTTP de 
# respuesta o Status de peticion_bicicleta:


Ahora puedes mirar [aquí](https://es.wikipedia.org/wiki/Anexo:C%C3%B3digos_de_estado_HTTP) si el código HTTP de respuesta significa algo bueno o algo malo (es decir: si todo ha salido bien o no).

Si ha salido bien, podemos seguir.

Bueno: resulta que una página web no deja de ser código. A pesar de que nosotros vemos la web de forma bonita y llena de imágenes y texto legible, las páginas están escritas en principalmente tres lenguajes: [HTML](https://es.wikipedia.org/wiki/HTML), [CSS](https://es.wikipedia.org/wiki/Hoja_de_estilos_en_cascada) y [Javascript](https://es.wikipedia.org/wiki/JavaScript).

Cuando pedimos una página web con nuestro navegador, éste recibe el código de la página. Una vez recibido, el nevagador "dibuja" o *renderiza* ese código para mostrarnos las páginas web bonitas y radiantes.

`Requests` no hace este útlimo paso: únicamente recibe el código de la página (llamado *código fuente*), con el que vamos a interactuar para hacer nuestro scraping.

Como prueba, vamos a hacer una cosa: vamos a abrir con nuestro navegador web favorito la [página de la bici](https://www.amazon.es/Moma-Bicicleta-Mountainbike-aluminio-suspensi%C3%B3n/dp/B00VXE2JCY/ref=lp_2929469031_1_1?s=sports&ie=UTF8&qid=1465237289&sr=1-1) y vamos a hacer click derecho, por ejemplo, en el título del artículo (*Bicicleta Montaña Mountainbike 26" BTT SHIMANO, aluminio, doble disco y doble suspensión*), y hacemos click en *Inspeccionar elemento*. Debería desplegarse una ventanita donde aparece el código fuente (en HTML principalmente) de esa parte de la página web.

Asimismo, algunos navegadores web nos permiten ver todo el código fuente de la página web en la que estamos: haciendo click derecho en cualquier sitio de ésta es posible que aparezca una opción llamada *Ver código fuente de la página* o algo similar.

`Requests` recibe el código fuente en lo que se llama *el contenido de la respuesta* tras la petición HTTP GET. Igual está disponible con algún atributo/método en `peticion_bicicleta`...

In [ ]:
# Imprime el código fuente de la página web
# de la bici. Va a ser largo, pero no pasa nada.
# Es posible que aparezca muchísimo espacio en
# blanco, pero no pasa nada: el código fuente
# de la página es el que es.
# Para ello, crea una nueva variable llamada
# codigo_fuente_bicicleta, y asígnale el 
# valor del código fuente recogido por Requests:


Ahí está todo: todo el texto, todas las imágenes... Todo. De ahí ahora seleccionaremos las partes que nos interesan, que van a ser los comentarios del producto.

## Preciosa sopa

Vamos a ver el tipo de dato que es `codigo_fuente_bicicleta`:

In [ ]:
# Haz un print del tipo de dato de
# codigo_fuente_bicicleta:


Un string de Python normal y corriente. Eso está bien: probablemente sea fácil extraer la información que queremos de ese caos de código HTML, CSS y Javascript...

... Sí lo es. Afortunadamente, existe una biblioteca muy famosa para hacer web scraping llamada [`BeautifulSoup`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/). `BeautifulSoup` cogerá este código fuente y nos permitirá extraer lo que necesitamos de forma sencilla.

`BeautifulSoup` es una biblioteca de terceros, así que será necesario instalarla. No obstante, de nuevo: Anaconda la trae instalada para nosostros. Así que podemos empezar a usarla directamente. Para ver cómo importarla igual te viene bien mirar un poco el [*quickstart*](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#quick-start) que los desarrolladores de la biblioteca han preparado...

In [ ]:
# Importa BeautifulSoup


Para que `BeautifulSoup` sea capaz de "analizar por nosotros" el código de la web es necesario crear una instancia de la clase `BeautifulSoup`. Dicha instancia debe ser creada con dos argumentos posicionales:

1. El primero deber ser el código fuente de la web en string (¡lo tenemos!)
2. El segundo ha de ser un string que le diga a `BeautifulSoup` el *parser* (o procesador) a utilizar. La biblioteca puede utilizar [unos cuantos](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser). Nosotros utilizaremos el `"lxml"`.

Manos a la obra.

In [ ]:
# Crea una instancia de BeautifulSoup
# llamada bici_bs:


`bici_bs` es una instancia de `BeautifulSoup`, lista para "darnos lo que le pidamos" de todo el código fuente de la página web.

Ahora solo queda decirle qué queremos...

Esta parte es más complicada si no sabes HTML, así que iremos despacio:

HTML es un lenguaje de marcas. Los elementos que podemos ver en una página web (como son títulos, párrafos, imágenes...) tienen nombres definidos. Por ejemplo:

+ Los títulos grandes son `h1`
+ Un párrafo es `p`
+ Una imagen es `img`

La tónica es casi siempre la misma: en HTML debemos "meter" el contenido de un elemento entre *marcas*. Por ejemplo, para hacer un título sería:

```html
<h1>Esto es un título.</h1>
```

Ponemos `<h1>` al principio, para indicar que comienza un título; y `</h1>` al final para decir que ahí termina. Otro elemento común son los `<div>`, que permiten "agrupar" varios elementos en un bloque. Por ejemplo:

```html
<div>
    <h1>Esto es un título</h1>
    <p>Y esto un párrafo. Y ambos están dentro de un div</p>
</div>
```

A los señores desarrolladores de la web de Amazon España les gusta mucho utilizar otro elemento de HTML llamado `<span>`, que es similar a `<p>`. Lo veremos cuando analicemos el código fuente de los comentarios de la bici.

Los elementos de HTML pueden tener dos cosas llamadas `id` y `class`. Esto permite a los propios desarrolladores poner algo de orden dentro de lo que es el caos de las páginas web modernas. Por ejemplo: una página como la de la bici tiene cientos de `<span>` y párrafos...

Pues bien, cada elemento puede (o debería al menos) tener un `id` único si queremos que sea diferenciable del resto de elementos del mismo tipo. Gracias a esto, podemos diferenciar (tanto nosotros como los ordenadores):

```html
<p id="comentario_1">Primer comentario</p>
```

De:

```html
<p id="comentario_2">Otro comentario</p>
```

Asimismo, elementos "similares" o del mismo estilo/aspecto suelen tener una misma `class` (no confundir con las clases de Python):

```html
<h1 class="titulo_grande">Un título</h1>
```
Y: 

```html
<h1 class="titulo_grande">Otro título</h1>
```

Pueden ser seleccionados fácilmente diciéndole a `BeautifulSoup`: *quiero todos los elementos de tipo `h1` que sean de `class="título_grande"`*.

Vamos a nuestro navegador favorito a "analizar a mano" un poco del código HTML de la página de la bici. Nos vamos a la sección de *Principales opiniones de clientes*, y hacemos click derecho justo en esa frase. Cuando se despliegue, hacemos click en *Inspeccionar elemento*.

En la consolita que se abre, podremos ver que ese *Principales opiniones de clientes* es un elemento de HTML tal que:

```html
<h3 class="a-spacing-small">Principales opiniones de clientes</h3>
```

Es un título de tamaño medianito (`h3` es más pequeñito que `h1`), y de `class` `"a-spacing-small"`. No sabemos muy bien lo que quiere decir ese nombre de clase, pero su uso tendrá para los desarrolladores de la web...

Si miramos el elemento justo debajo de ese `h3` viene ya lo interesante: 

```html
<div class="a-row a-spacing-large">
```

Si nuestro navegador nos "subraya" qué es cada elemento, podrás ver que ese `div` engloba todos los comentarios; que es lo que buscamos... ¡Bingo!

Me pregunto si será también así en la página de otros productos de Amazon... Vamos a ver en otra al azar: por ejemplo [esta](https://www.amazon.es/WD-Elements-Disco-externo-port%C3%A1til/dp/B00CRZ2PRM/ref=lp_937903031_1_2?s=computers&ie=UTF8&qid=1465243368&sr=1-2).

Pues también. Exactamente igual. `<div class="a-row a-spacing-large">` parece ser constante en los comentarios de todos los productos.

Nosotros ya hemos trabajado. Ahora es el turno de BeautifulSoup. Resulta que cualquier instancia de `BeautifulSoup` tiene el método [`.find()`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find) que permite buscar cualquier tipo de elemento HTML, y se queda con la primera ocurrencia. Este método toma como argumentos interesantes:

1. Un posicional, que es un string con el nombre del elemento a buscar (como pueden ser `"h1"`, `"p"` o `"div"`).
2. Un argumento opcional llamado `id`, que casualmente permite especificar más en la búsqueda; buscando solo elementos con el `id` especificado como string.
3. Otro argumento opcional llamado `class_` (así con barra baja para diferenciarlo de la keyword `class` que sirve para definir clases en Python). Al igual que `id`, permite limitar la búsqueda a los elementos del HTML que tengan dicha `class`.

Por ejemplo: si queremos buscar el primer elemento `h2` que tenga `id="123"` y `class="front-title-spacing"`, haríamos:

```python
mi_instancia_de_BeatutifulSoup.find("h2", id="123", class_="front-title-spacing")
```

Probemos pues a buscar:

```html
   
<div class="a-section review-views celwidget">
```

En nuestra instancia `bici_bs`:

In [ ]:
# Utiliza el método .find() para 
# buscar el elemento de HTML que buscamos,
# que es el div que contiene los comentarios
# del producto. Guarda el resultado en una
# variable llamada bici_comentarios:




¿Qué tipo de dato será `bici_comentarios`?

In [ ]:
# Imprime el tipo de dato de
# bici_comentarios:


Es un `Tag` de `BeautifulSoup`. Básicamente: la versión ya filtrada del código fuente de la página, que solo contiene la parte que queremos.

Vamos a hacer un `print()` de `bici_comentarios`; a ver qué sale...

Todo el contenido de ese `div`, de principio a fin. Hemos filtrado mucho; pero aún nos queda para tener los datos que nos interesan.

En cada uno de los comentarios queremos quedarnos con:

+ El número de estrellas que el usuario le ha dado a la bici.
+ El texto del comentario como tal. 

En este ejercicio vamos a ignorar el resto de información (nombre del autor del comentario, fecha y demás). ¡Siéntete libre de intentar extraerlos!

Para obtenerlos nos va a tocar volver a inspeccionar manualmente la página web. Cogemos nuestro navegador web de nuevo, y miramos la línea justo debajo de la del `div` que contiene todas las reviews (la que hemos extraído). Resulta que esta línea es:


```html
<div id="R2L4SORBKQBFYW" class="a-section review aok-relative">
```

Y contiene toda la primera review. 

Tras un poco más de exploración más hacia abajo en el HTML, veremos que la segunda review empieza así:

```html
<div id="R170P7SFNFRTQK" class="a-section review aok-relative">
```

Y la tercera:

```html
<div id="R1WXYTOUKC8OJ8" class="a-section review aok-relative">
```

Podemos ver un patrón claro:


+ La `class` de cada uno es siempre la misma: `"a-section review aok-relative"`

Utilizar este patrón debería ser sencillo. Si sale bien, podremos obtener todas las reviews ya de forma individual (separadas unas de otras).

Además de `.find()` (que solo busca la primera ocurrencia que encaje), las instancias tanto de `BeautifulSoup` como de `Tag` tienen el método `.find_all()`. Este método funciona exactamente igual que `.find()`, pero con la diferencia de que busca todos los elementos que satisfacen la búsqueda; y los devuelve en una lista de Python.

Probemos entonces a buscar en función del segundo patrón.

In [ ]:
# Utiliza el método .find_all() sobre
# bici_comentarios para obtener una lista
# con todos div con un comentario cada uno,
# y llama a dicha lista bici_comentarios_lista:
<tu-codigo-aqui>



La página de la bici tiene 8 comentarios (como podrás ver en tu navegador). Dependiendo de cuándo hagas la práctica, es posible que tenga más (o menos, si alguien ha borrado alguno) ¿Qué longitud tiene nuestra `bici_comentarios_lista`?

In [ ]:
# Comprueba la longitud de 
# bici_comentarios_lista:
<tu-codigo-aqui>


Buena señal. Pero aún tenemos que filtrar más en cada uno de esos comentarios...

Misma historia. Toca hacer trabajo manual. Queremos extraer el número de estrellas y el cuerpo del comentario. Comencemos por las estrellas.

Vamos al navegador y hacemos click derecho en las estrellitas, e inspeccionamos elemento. Veremos que por ahí pone:

```html
<span class="a-icon-alt">5,0 de 5 estrellas</span>
```

Y si miramos otras reviews, veremos que el patrón es constante: podemos sacar ese texto buscando elementos de tipo `span` con `class=a-icon-alt`. De nuevo, con el método `.find()` debería bastar. Eso sí: tenemos que hacerlo en cada uno de los comentarios. 

¿Aplicar la misma acción o transformación a todos los elementos de una colección/iterable? Me suena eso...

In [ ]:
# Quizás puedas usar una función map() o
# una list comprehension para extraer ese 
# elemento de nuestras reviews, que están en
# la lista bici_comentarios_lista... O igual un for...
# Mete el resultado en una variable que se llame
# estrellas_bici_lista:
<tu-codigo-aqui>


Probemos a imprimir `estrellas_bici_lista`:

In [ ]:
# Imprime estrellas_bici_lista:
<tu-codigo-aqui>


Tenemos las estrellas. Pero nos sobra cosa ahí: queremos quedarnos únicamente con las estrellas, como números `float` (porque así lo deseo). Todo el demás texto nos sobra.

Una vez que hemos aislado con `BeautifulSoup` el elemento que queremos, podemos utilizar el método `.get_text(" ")` para quedarnos con el texto en bruto (le pasamos como argumento un string con un espacio en blanco para que nos separe bien los párrafos). De nuevo, queremos hacerlo para las estrellas de cada una de las reviews...

In [ ]:
# Aplica .get_text() en todos los elementos de
# estrellas_bici_lista, y el resultado guárdalo
# en una variable llamada estrellas_bici_lista_solotexto:
<tu-codigo-aqui>

# E imprime dicha variable:
<tu-codigo-aqui>


Casi las tenemos. Solo nos queda quitar el `"de 5 estrellas"` y sustituir la coma por punto. Hacer slicing de cada string, y sustituir coma por punto podría ser la solución. Con hacer un slice desde el inicio hasta el cuarto caracter + un `.replace` debería valer...

In [ ]:
# Aplica una transformación en 
# cada elemento de estrellas_bici_lista_solotexto,
# que se haga el slicing adecuado y reemplazo de coma
# por punto para que se quede
# solo con el número (por ejemplo: "5.0").
# Asigna el resultado a una variable que se llame
# numero_estrellas_bici_lista:
<tu-codigo-aqui>

# E imprime el resultado:
<tu-codigo-aqui>


Tenemos las estrellas. Gracias a que las listas de Python son colecciones ordenadas, sabemos que el primer número de estrellas es del primer comentario, el segundo del segundo, etcétera.

Ahora, misma historia para el cuerpo del texto de cada comentario. Analizar manualmente, extraer el patrón que permita a `BeautifulSoup` buscar lo que queremos, y aplicar los `.find`, `.find_all`, `.get_text()` y demás transformaciones necesarias para extraer el texto.

Recuerda que en `bici_comentarios_lista` tenemos ya una lista con el cuerpo entero de todo lo que hay en cada comentario. Queda filtrar de nuevo a partir de ahí; solo que buscando el texto de cada comentario y no las estrellas.

In [ ]:
# ¡Tu turno! Consigue quedarte con el
# texto de cada review. Utiliza tantas
# líneas de código como necesites.
# Si ves saltos de línea \n en tu resultado
# no te preocupes: luego los limpiaremos.
# Guarda el resultado en una lista (que debería
# tener 8 elementos) llamada texto_comentarios_bici_lista:

<tu-codigo-aqui>


In [ ]:
# Imprime texto_comentarios_bici_lista:
<tu-codigo-aqui>

# E imprime su longitud:
<tu-codigo-aqui>


Ahora vamos a quitar los posibles `"\n"` de salto de línea que pueda haber. A veces salen al principio y al final de cada comentario. Resulta que las strings tienen el método [`.strip()`](https://docs.python.org/3/library/stdtypes.html#str.strip) para estos propósitos :)

In [ ]:
# Aplica .strip() a cada comentario
# de texto_comentarios_bici_lista,
# y guarda el resultado en una variable
# llamada texto_comentarios_bici_lista_limpio:
<tu-codigo-aqui>

# E imprime el resultado:
<tu-codigo-aqui>


¡Bien! Tenemos nuestros resultados en dos listas: en una de ellas las estrellas, y en la otra los comentarios. Molaría ahora "unir" ambas listas, teniendo en cuenta que el primer elemento de cada una de ellas es el del primer comentario, el segundo del segundo comentario... 

El objetivo ahora es conseguir una sola lista, donde cada elemento es una tupla que contiene las estrellas y el texto de cada review. ¿Qué función vista en la teoría podríamos usar?

In [ ]:
# Une ambas listas en una sola,
# que sea una lista de tuplas estilo
# (estrellas, comentario). Guarda el
# resultado en una variable llamada
# estrellas_y_comentarios_bici:
<tu-codigo-aqui>

# E imprime el resultado:
<tu-codigo-aqui>


## La vida es más sencilla si metes el código en funciones

Y terminamos. Hemos escrito bastante código para *scrapear* los comentarios y las estrellas de cada review. Estaría bien definir una función que automatizara el proceso. Con un poco de suerte, permitiría scrapear y obtener los resultados directamente de cualquier producto de Amazon España...

Así que vamos a definir una función llamada `scrapear_pagina_amazon`, que toma como argumento posicional `link` (que será el string del link de una página de producto de Amazon), y que devuelve una lista con los pares de estrella-comentario; tal y como hemos hecho para la bici.

In [ ]:
# ¡Adelante! Siéntete libre de reutilizar
# tanto código del que has ido escribiendo
# hasta ahora como desees (aunque igual cambiando
# el nombre de las variables para que no sea
# siempre bici_esto, bici_lo_otro):
<tu-codigo-aqui>

Prueba tu función en el link de las bicis (para ver si el resultado es el mismo que obtuviste sin la función):

In [ ]:
# ¡Llama a la función y mira si el resultado es el mismo!
<tu-codigo-aqui>

Hemos construido una función que scrapea páginas web de Amazon. Siéntete libre de probar un par más de páginas de productos (que tengan comentarios), a ver si también funciona:

In [ ]:
# Llama aquí a la función pasando como argumento otro link de otro
# producto de Amazon España:
<tu-codigo-aqui>

In [ ]:
# Y con otro producto distinto más:
<tu-codigo-aqui>

El scraping web es una herramienta poderosa (y tediosa; todo sea dicho, debido a que páginas como Amazon intentan protegerse de él) para obtener datos de Internet cuando no están disponibles en bases de datos o APIs fáciles de usar. De hecho [hay empresas que se dedican exclusivamente a esto](http://scrapinghub.com/); y las herramientas que utilizan son exactamente las mismas que has utilizado en este ejercicio (`Requests` y `BeautifulSoup`); aunque hay una más...

[`Scrapy`](http://scrapy.org/) es un scraper igual que `BeautifulSoup`, pero también es un [*web crawler*](https://en.wikipedia.org/wiki/Web_crawler) o *web spider*. 

Un *crawler* es un programa que no solo extrae aquello que queremos de una página (scraping), sino que también se encarga de buscar las propias páginas. Por ejemplo: a un crawler escrito con `Scrapy` le podríamos pasar un dominio (como puede ser Amazon.es), y él solo se encargaría de "meterse" en todas las páginas de productos de Amazon, utilizando luego `BeautifulSoup` para *scrapear* y extraer los datos que queremos de cada una de ellas.

`Scrapy` es una biblioteca algo más complicada. Pero teniendo su [documentación](http://doc.scrapy.org/en/latest/) y algo de paciencia, no deberías tener demasiados problemas para conseguir tus propósitos :)